
# Recommendation for Neuron TP and server batch tuning

This is intended to assist engineers in optimizing the inference performance of LLMs by tuning Tensor Parallelism (TP) and batch size parameters for AWS Inf2 and Trn1 instances. The guidance provided here will help you understand the total memory requirements for a LLM and how it affects the TP and batch size configuration. 

Transformer models are typically defined via a hyper-parameter configuration, such as the following:

```
"n_ctx": 2048, # maximum possible sequence length of the input
"hidden_size": 9216, # refers to the dimensionality of the vectors that represent each token
"attention_heads": 72, # number of attention heads (hidden_size must be divisible by attention_heads)
"n_layer": 64 # number of layers (determines the "depth" of the network)
"dtype_size" = 2 # Data type size (example 2 for BF16/FP16, 1 for FP8/INT8)
```

To determine the number of Neuron Cores needed to fit the model, we need to perform the following calculation to understand the memory footprint of a LLM:

**weight_mem_footprint = 12 x <n_layer> x <hidden_size>^2 x <dtype-size>**

**KV_cache_mem_footprint = <batch-size> x <n_layer> x <n_ctx> x <hidden_size> x 2 x <dtype-size>**

**mem_footprint = weight_mem_footprint + KV_cache_mem_footprint**

#### For example for OPT-66B example (BF16, Inf2) these values are as follows:


n_layer=64,
n_ctx=2048,
hidden_size=9216,
batch=16 # user configuration dependinng on performance needs


weight_mem_footprint = 12 x 64 x 9216^2 x 2 = 121.5 GiB,
KV_cache_mem_footprint = 16 x 64 x 2048 x 9216 x 2 x 2 = 72 GiB,
mem_footprint = 121.5GiB + 72GiB = 193.5 GiB


Note: Currently, the Neuron runtime supports tensor-parallelism degrees 2, 8, and 32 on Trn1 and supports tensor-parallelism degrees 2, 4, 8, 12 and 24 on Inf2.

Therefore for the above OPT-66B example 

On Inf2 instance: num_neuron_cores (TP)= ceil_to_closest_supported_size (193.5GiB / 16GiB) = (12.1) = 24
On Trn1 instance: num_neuron_cores (TP)= ceil_to_closest_supported_size (193.5GiB / 16GiB) = (12.1) = 32


**To determine the appropriate configuration for serving a large language model (LLM) on AWS Inf2 and Trn1 instances, we first calculate the memory footprint required for the LLM. This calculation provides a theoretical maximum batch size that can be accommodated on the Neuron accelerators with tensor parallelism (TP). However, in practice, the actual batch size will need to be slightly smaller to ensure the model compiles and does not run out of memory. The process involves starting with a smaller batch size, and then iteratively increasing it to find the largest size that fits the Neuron accelerators without exceeding the available memory. This guidance on batch size and TP configurations will help identify the optimal configurations for efficiently serving the LLM on the AWS Inf2 and Trn1 instances.**

In [4]:

# Define model and system parameters
# The below pre-filled example is for Llama2 13B model

attention_heads = 40   # Number of attention heads
n_layer = 40   # Number of layers
hidden_size = 5120   # Size of embeddings
dtype_size = 2 # Data type size (example 2 for BF16/FP16, 1 for FP8/INT8)
n_ctx = 8192    # Max Context size for your use case

# Choose the memory options based on the instance type Inf2 or Trn1.
# For Trn1 currently, the Neuron runtime supports tensor-parallelism degrees 2, 8, and 32.
# For Inf2 currently, the Neuron runtime supports tensor-parallelism degrees 2, 4, 8, 12 and 24.

memory_options_gb = {32: 'TP 2', 64: 'TP 4', 128: 'TP 8', 192: 'TP 12', 384: 'TP 24'}  # Inf2 Memory options in GB
#memory_options_gb = {32: 'TP 2', 128: 'TP 8', 512: 'TP 32'}  # Trn1 Memory options in GB




In [5]:
def calculate_optimal_batch_size(n_layer, hidden_size, dtype_size, n_ctx, memory_options):
    # Calculate constant weight memory footprint in bytes
    weight_mem_footprint_bytes = 12 * n_layer * hidden_size**2 * dtype_size

    # Function to calculate KV cache memory footprint for a given batch size in bytes
    def KV_cache_mem_footprint(batch_size):
        return batch_size * n_layer * n_ctx * hidden_size * 2 * dtype_size

    # Convert memory footprint from bytes to gigabytes
    def bytes_to_gb(bytes):
        return bytes / (2**30)

    optimal_batch_sizes = {}
    for memory_gb, tp_label in memory_options.items():
        memory_bytes = memory_gb * (2**30)  # Convert GB to bytes
        batch_size = 0
        current_footprint = weight_mem_footprint_bytes + KV_cache_mem_footprint(batch_size)
        while current_footprint <= memory_bytes:
            batch_size += 1
            current_footprint = weight_mem_footprint_bytes + KV_cache_mem_footprint(batch_size)
        
        # Get the last valid batch size and its memory footprints
        last_valid_batch_size = batch_size - 1
        used_KV_cache_gb = bytes_to_gb(KV_cache_mem_footprint(last_valid_batch_size))
        weight_mem_footprint_gb = bytes_to_gb(weight_mem_footprint_bytes)
        total_used_memory_gb = used_KV_cache_gb + weight_mem_footprint_gb
        optimal_batch_sizes[tp_label] = (last_valid_batch_size, total_used_memory_gb, memory_gb, used_KV_cache_gb, weight_mem_footprint_gb)

    return optimal_batch_sizes


# Calculate and display optimal batch sizes and memory footprints
optimal_batch_sizes = calculate_optimal_batch_size(n_layer, hidden_size, dtype_size, n_ctx, memory_options_gb)
for tp_label, (batch_size, total_used_gb, total_memory_gb, used_KV_cache_gb, weight_mem_footprint_gb) in optimal_batch_sizes.items():
    print(f"{tp_label}: {batch_size} batches, total used memory {total_used_gb:.2f} GB of {total_memory_gb} GB available")
    print(f"  - KV Cache Memory Footprint: {used_KV_cache_gb:.2f} GB")
    print(f"  - Weight Memory Footprint: {weight_mem_footprint_gb:.2f} GB")


TP 2: 1 batches, total used memory 29.69 GB of 32 GB available
  - KV Cache Memory Footprint: 6.25 GB
  - Weight Memory Footprint: 23.44 GB
TP 4: 6 batches, total used memory 60.94 GB of 64 GB available
  - KV Cache Memory Footprint: 37.50 GB
  - Weight Memory Footprint: 23.44 GB
TP 8: 16 batches, total used memory 123.44 GB of 128 GB available
  - KV Cache Memory Footprint: 100.00 GB
  - Weight Memory Footprint: 23.44 GB
TP 12: 26 batches, total used memory 185.94 GB of 192 GB available
  - KV Cache Memory Footprint: 162.50 GB
  - Weight Memory Footprint: 23.44 GB
TP 24: 57 batches, total used memory 379.69 GB of 384 GB available
  - KV Cache Memory Footprint: 356.25 GB
  - Weight Memory Footprint: 23.44 GB


In [6]:

# Check divisibility of TP values by the number of attention heads and print recommendations
print("Recommendation based on TP divisibility by number of attention heads:")
for memory_gb, tp_str in memory_options_gb.items():
    # Extract the numerical part from the TP string
    tp_value = int(tp_str.split()[1])
    
    if attention_heads % tp_value == 0:
        recommendation = "Recommended"
    else:
        recommendation = "Will not work as attn heads not divisible by TP value"
    
    print(f"TP {tp_value} (GPU memory {memory_gb} GB): {recommendation}")




Recommendation based on TP divisibility by number of attention heads:
TP 2 (GPU memory 32 GB): Recommended
TP 4 (GPU memory 64 GB): Recommended
TP 8 (GPU memory 128 GB): Recommended
TP 12 (GPU memory 192 GB): Will not work as attn heads not divisible by TP value
TP 24 (GPU memory 384 GB): Will not work as attn heads not divisible by TP value
